In [16]:
from hdf5libs import HDF5RawDataFile

import daqdataformats
import detdataformats
from daqdataformats import FragmentType
from rawdatautils.unpack.daphne import *

import matplotlib.pyplot as plt

In [21]:
#filename  = "/data0/np04hd_raw_run025016_0000_dataflow4_datawriter_0_20240415T141456.hdf5.copied"
filename  = '/data0/np04hd_raw_run025077_0213_dataflow0_datawriter_0_20240419T092425.hdf5.copied' 
run       = filename.split('_')[2]
h5_file   = HDF5RawDataFile(filename)
records   = h5_file.get_all_record_ids()
nrecords  = len(records)
nskip     = 1
det       = 'HD_PDS'

In [22]:
### Reading HDF5 files and saving the important information in a vector
info_perfragment = []

for r in records[nskip:nrecords]:
    
    pds_geo_ids    = list(h5_file.get_geo_ids_for_subdetector(r,detdataformats.DetID.string_to_subdetector(det)))
    for gid in pds_geo_ids:

        frag = h5_file.get_frag(r,gid)
        frag_id    = str(frag).split(' ')[3][:-1]
        fragType = frag.get_header().fragment_type
        
        # For self trigger
        if fragType == FragmentType.kDAPHNE.value:    
            timestamps = np_array_timestamp(frag)
            adcs       = np_array_adc(frag)
            channels   = np_array_channels(frag)

        # For full_stream
        elif fragType == 13:
            timestamps = np_array_timestamp_stream(frag)
            adcs       = np_array_adc_stream(frag)
            channels   = np_array_channels_stream(frag)[0]

        info_perfragment.append([frag_id, gid, channels, adcs, timestamps])
        

In [23]:
#### Persistence of each channel per geo_id

gid = list(set(item[1] for item in info_perfragment))

for g in gid:
    ch_ref = None
    wf_list = None
    match = 'no'

    for index, item in enumerate(info_perfragment):
        fragment, geo_id, channels, waveforms, timestamps = item
        
        if geo_id == g:
            if match == 'no':
                ch_ref = list(set(channels))
                wf_list = [[] for _ in range(len(channels))]
                match = 'yes'

            for i, wf in enumerate(waveforms):
                ch_index = ch_ref.index(channels[i])
                wf_list[ch_index].append(wf)

        if geo_id != g or index == (len(info_perfragment) - 1):
            match = 'no'
            if ch_ref is not None:
                for c, ch in enumerate(ch_ref):
                    for wf in wf_list[c]:
                        plt.plot(wf)
                    plt.title(f'ID: {g} Channel = {ch}')
                    plt.show()

In [24]:
### Waveforms per fragment & timestamps

# Get unique geo_ids
gid = list(set(item[1] for item in info_perfragment))

for index, item in enumerate(info_perfragment):
    fragment, geo_id, channels, waveforms, timestamps = item

    tm_ref = list(set(timestamps))
    for t_element in tm_ref:
        indices = [i for i, tm in enumerate(timestamps) if tm == t_element]

        for i in indices:
            plt.plot(waveforms[i], label=channels[i])

        plt.title(f'Fragment: {fragment}; Timestamp: {t_element}')
        plt.legend()
        plt.show()